<a href="https://colab.research.google.com/github/2019mohamed/ArabDialectClassification/blob/master/Transformer_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 32.9 MB/s 
     |████████████████████████████████| 6.5 MB 16.3 MB/s 
     |████████████████████████████████| 67 kB 4.3 MB/s 
     |████████████████████████████████| 895 kB 22.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from torch import nn
from torch.optim import Adam

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/two_clo_dialect_dataset.csv')
df

,new_text,label
0,بالنهايه ينتفض يغير,0
1,يعني محسوب البشر حيونه ووحشيه وتطلبون الغرب يح...,0
2,مبين كلامه خليجي,0
3,يسلملي مرورك وروحك الحلوه,0
4,وين الغيبه اخ محمد,0
...,...,...
458192,مبسوطين منك اللي باسطانا,17
458193,والله ماينده ابش يختي,17
458194,شو عملنا حنا تهربي مننا احنا مساكين ليش بتعملي...,17
458195,الله يبارك وبالعافيه,17


In [ ]:
corpus = df['new_text'].astype('str').tolist()
labels = df['label'].tolist()

In [ ]:
def get_index ():
  freqs = df['label'].value_counts()

  start = [0 for _ in range(18)]
  start[0] = 0
  for c in range(1,18):
      start[c] += freqs[c-1] + start[c-1]  
  indexes = []# (mini , maxi+1) for all classes
  for c in range(18):
      indexes.append(  (start[c] , start[c] + freqs[c]) )
    
  return indexes

In [ ]:
def sample_data (data , label ,intervals, lim = 1000):
  data = np.array(data)
  label = np.array(label)
  ret = np.array([])
  retl = np.array([])
  for i in range(18):
    l = intervals[i][0]
    h = intervals[i][1]


    index = np.random.choice(range(l,h), size = lim , replace = False)


    ret = np.concatenate((ret , data[index]), axis = 0)
    retl = np.concatenate((retl , label[index]), axis = 0)

  return ret,retl

In [ ]:
indexes = get_index()

In [ ]:
sample_corpus, sample_label = sample_data(corpus , labels, indexes , lim = 2000)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('asafaya/bert-base-arabic')

Downloading:   0%|          | 0.00/326k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/491 [00:00<?, ?B/s]

In [ ]:
class SeqData(torch.utils.data.Dataset):

    def __init__(self):

        self.labels = sample_label
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 64, truncation=True,
                                return_tensors="pt") for text in sample_corpus]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [ ]:
if torch.cuda.is_available():
  device = torch.device('cuda:0')
  print('GPU')
else:
  device = torch.device('cpu')
  print('CPU')

GPU


In [ ]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('asafaya/bert-base-arabic')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 18)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [ ]:
from torch.utils.data.sampler import SubsetRandomSampler , BatchSampler
from torch.utils.data import Dataset, DataLoader

In [ ]:
def split_rand(dataset,batch_size, split_ratio=0.7, seed=42, shuffle=True):
    import math
    num_entries = len(dataset)
    indices = list(range(num_entries))
    np.random.seed(seed)
    np.random.shuffle(indices)
    split = int(math.floor(split_ratio * num_entries))
    train_idx, valid_idx = indices[:split], indices[split:]
    
    
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = DataLoader(
            dataset, sampler=train_sampler,
            batch_size=batch_size)
    
    valid_loader = DataLoader(
            dataset, sampler=valid_sampler,
            batch_size=batch_size)

    return train_loader, valid_loader

In [ ]:
def train(net, trainloader, optimizer, criterion):
    net.train()

    running_loss = 0
    total_iters = len(trainloader)

    for idx , data  in enumerate(trainloader):

        x, labels= data
        labels = labels.type(torch.LongTensor)
        labels = labels.to(device)
        #print(x)
        mask = x['attention_mask'].to(device)
        input_id = x['input_ids'].squeeze(1).to(device)
        outputs = net(input_id , mask)

        loss = criterion(outputs, labels)
        running_loss += loss.item()

        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    running_loss = running_loss / total_iters

    return running_loss

In [ ]:
def eval_net(net, dataloader, criterion):
    net.eval()

    total = 0
    total_loss = 0
    total_correct = 0

    for idx ,  data in enumerate(dataloader):
        x, labels = data
        labels = labels.type(torch.LongTensor)
        
        labels = labels.to(device)

        mask = x['attention_mask'].to(device)
        input_id = x['input_ids'].squeeze(1).to(device)

        total += len(labels)
        outputs = net(input_id , mask)
        _, predicted = torch.max(outputs.data, 1)

        total_correct += (predicted == labels.data).sum().item()
        loss = criterion(outputs, labels)
        total_loss += loss.item() * len(labels)

    loss, acc = 1.0*total_loss / total, 1.0*total_correct / total


    return loss, acc

In [ ]:
dataset = SeqData()

In [ ]:
train_loader , test_loader = split_rand(dataset, batch_size = 2)

In [ ]:
model = BertClassifier().to(device)

Some weights of the model checkpoint at asafaya/bert-base-arabic were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import torch.optim as optim

In [ ]:
criterion = nn.CrossEntropyLoss()  
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
train_loss = []
test_loss = []
test_acc = []

In [ ]:
for _ in range(5):
    tloss = train(model, train_loader, optimizer, criterion)
    train_loss.append(tloss)
    print(tloss)
    
    eloss, eacc= eval_net(model, test_loader, criterion)
    test_loss.append(eloss)
    test_acc.append(eacc)
    print(eloss,' ',eacc)

2.8912022206518384
2.890371799468994   0.05694444444444444
2.890371799468994
2.890371799468994   0.05694444444444444
2.890371799468994
2.890371799468994   0.05694444444444444
2.8903736175431147
2.890371799468994   0.05694444444444444
2.890371799468994


In [ ]:
torch.save(model.state_dict(), 'Transformer2.pth')